In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import nibabel as nib
import os
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

user = 'Cat'

if user=='Cat':
    project_home = '/Users/catcamacho/Box/CCP/HBN_study/'
elif user=='Emily':
    project_home = '/Users/emilyfurtado/Box/HBN_study/'

In [2]:
full_data = pd.read_csv(project_home + 'stress_analysis/full_data_mot0.8_20211013.csv',index_col=0)
stress = pd.read_csv(project_home + 'stress_analysis/complete_samp_data_20210825.csv',
                     index_col=0)[['NLES_P_parent_total', 'NLES_P_parent_upset', 'NLES_P_child_total', 'NLES_P_child_upset',
                                   'NLES_SR_parent_total', 'NLES_SR_parent_upset', 'NLES_SR_child_total', 'NLES_SR_child_upset']]
stress['sub'] = ['sub-' + a for a in stress.index]
stress['subject_id'] = stress.index
stress.index = stress['sub']
data = full_data.merge(stress, how='left', left_index=True, right_index=True)
data.describe()

,age,female,meanFD,righthanded,L_None_1,L_None_2,L_None_3,L_None_4,L_None_5,L_None_6,...,net_SMmouth,net_Auditory,NLES_P_parent_total,NLES_P_parent_upset,NLES_P_child_total,NLES_P_child_upset,NLES_SR_parent_total,NLES_SR_parent_upset,NLES_SR_child_total,NLES_SR_child_upset
count,1978.000000,1978.000000,1978.000000,1978.000000,1978.000000,1978.000000,1978.000000,1978.000000,1978.000000,1978.000000,...,1978.000000,1978.000000,1750.000000,1750.000000,1750.000000,1750.000000,596.000000,596.000000,596.000000,596.000000
mean,10.599110,0.408493,0.354991,0.771486,-0.022039,-0.006102,-0.012215,0.014362,0.012298,0.003605,...,-0.013899,-0.001154,3.052571,8.582857,2.404571,7.494857,3.681208,6.942953,3.550336,8.218121
std,2.835783,0.491680,0.181142,0.419981,0.140711,0.128476,0.124404,0.171076,0.191981,0.144563,...,0.082208,0.045356,2.124103,7.266923,1.541046,5.940695,3.832946,7.599653,2.700604,6.417105
min,5.055213,0.000000,0.077657,0.000000,-0.601625,-0.636636,-0.554453,-0.639684,-0.620939,-0.582025,...,-0.361759,-0.194685,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.379534,0.000000,0.210463,1.000000,-0.111723,-0.087352,-0.083851,-0.093809,-0.107721,-0.090118,...,-0.057527,-0.027868,1.000000,3.000000,1.000000,3.000000,1.000000,1.000000,1.000000,3.000000
50%,10.274013,0.000000,0.313953,1.000000,-0.016850,-0.003042,-0.004691,0.015247,0.013775,0.007192,...,-0.009148,0.000208,3.000000,7.000000,2.000000,6.000000,3.000000,5.000000,3.000000,7.000000
75%,13.099018,1.000000,0.480957,1.000000,0.068194,0.077542,0.068309,0.124648,0.133750,0.099467,...,0.034846,0.027787,4.000000,12.000000,3.000000,10.000000,5.000000,10.000000,5.000000,12.000000
max,15.987451,1.000000,1.001800,1.000000,0.587605,0.428923,0.426900,0.664616,0.730983,0.521193,...,0.311718,0.210836,11.000000,45.000000,8.000000,35.000000,18.000000,40.000000,11.000000,30.000000


In [3]:
# parent-reported
bx_data = data[['NLES_P_parent_total', 'NLES_P_parent_upset', 'NLES_P_child_total', 'NLES_P_child_upset',
                'NLES_SR_parent_total', 'NLES_SR_parent_upset', 'NLES_SR_child_total', 'NLES_SR_child_upset', 
                'MFQ_SR_Total','MFQ_P_Total','age','female','site']].drop_duplicates()

discovery = bx_data.loc[bx_data['site']=='cbic',:]
replication = bx_data.loc[bx_data['site']=='rubic',:]

print('####### Discovery #######')
res = smf.glm('MFQ_P_Total ~ NLES_P_parent_upset + age + female', data=discovery).fit()
print(res.summary())

res = smf.glm('MFQ_P_Total ~ NLES_P_child_upset + age + female', data=discovery).fit()
print(res.summary())

print('####### Replication #######')
res = smf.glm('MFQ_P_Total ~ NLES_P_parent_upset + age + female', data=replication).fit()
print(res.summary())

res = smf.glm('MFQ_P_Total ~ NLES_P_child_upset + age + female', data=replication).fit()
print(res.summary())

####### Discovery #######
                 Generalized Linear Model Regression Results                  
Dep. Variable:            MFQ_P_Total   No. Observations:                  272
Model:                            GLM   Df Residuals:                      268
Model Family:                Gaussian   Df Model:                            3
Link Function:               identity   Scale:                          60.246
Method:                          IRLS   Log-Likelihood:                -941.32
Date:                Thu, 14 Oct 2021   Deviance:                       16146.
Time:                        10:39:39   Pearson chi2:                 1.61e+04
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercep

In [4]:
# child-reported
res = smf.wls('MFQ_SR_Total ~ NLES_SR_parent_upset + age + female', data=discovery).fit()
print(res.summary())

res = smf.wls('MFQ_SR_Total ~ NLES_SR_child_upset + age + female', data=discovery).fit()
print(res.summary())

print('####### Replication #######')
# child-reported
res = smf.wls('MFQ_SR_Total ~ NLES_SR_parent_upset + age + female', data=replication).fit()
print(res.summary())

res = smf.wls('MFQ_SR_Total ~ NLES_SR_child_upset + age + female', data=replication).fit()
print(res.summary())

                            WLS Regression Results                            
Dep. Variable:           MFQ_SR_Total   R-squared:                       0.058
Model:                            WLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     1.789
Date:                Thu, 14 Oct 2021   Prob (F-statistic):              0.155
Time:                        10:39:39   Log-Likelihood:                -356.34
No. Observations:                  91   AIC:                             720.7
Df Residuals:                      87   BIC:                             730.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               13.8417 

In [ ]:
## LCA